## Setup

In [ ]:
# Import Libraries
import sqlite3
import pandas as pd
from datetime import datetime, timedelta

In [2]:
# Connect to the SQLite database
# Make sure the path is correct. Adjust the path as needed.
database = ('../data/processed/study_buddy.db')
conn = sqlite3.connect(database)

In [3]:
# Load tables into DataFrames

# Load students
df_students = pd.read_sql("SELECT * FROM students", conn)

# Load UTC study days
df_utc_days = pd.read_sql("SELECT * FROM utc_study_days", conn)



## Manual Scheduling Function

In [4]:
def create_manual_session(conn, host_id, guest_id, day, start_time, end_time):
    """
    Create a manually scheduled session by the host student.
    """
    cursor = conn.cursor()
    
    cursor.execute("""
        INSERT INTO scheduled_sessions (
            host_student_id,
            guest_student_id,
            day,
            start_time,
            end_time,
            status
        )
        VALUES (?, ?, ?, ?, ?, 'pending');
    """, (host_id, guest_id, day, start_time, end_time))
    
    # Optional: Create a notification
    cursor.execute("""
        INSERT INTO notifications (
            student_id, type, content, status, timestamp
        ) VALUES (?, ?, ?, ?, datetime('now'))
    """, (guest_id, 'invite', f"{host_id} invited you to a study session on {day} at {start_time}", 'unread'))
    
    conn.commit()
    print("Manual session created and notification sent.")


##  Smart Suggested Time Logic

In [5]:
def find_common_time_block(host, guest, df_students, df_utc_days):
    """
    Finds overlapping study time blocks for two students based on UTC.
    Returns a list of (day, start_time, end_time) tuples.
    """
    host_days = set(df_utc_days[df_utc_days["student_id"] == host]["utc_day"])
    guest_days = set(df_utc_days[df_utc_days["student_id"] == guest]["utc_day"])
    common_days = host_days & guest_days

    if not common_days:
        return []

    # Get time ranges
    host_row = df_students[df_students["student_id"] == host].iloc[0]
    guest_row = df_students[df_students["student_id"] == guest].iloc[0]
    
    host_start = datetime.strptime(host_row["utc_start_time"], "%H:%M")
    host_end = datetime.strptime(host_row["utc_end_time"], "%H:%M")
    
    guest_start = datetime.strptime(guest_row["utc_start_time"], "%H:%M")
    guest_end = datetime.strptime(guest_row["utc_end_time"], "%H:%M")

    # Calculate overlapping time range
    overlap_start = max(host_start, guest_start)
    overlap_end = min(host_end, guest_end)

    if overlap_start >= overlap_end:
        return []  # no time overlap

    # Return overlapping time for all common days
    result = []
    for day in common_days:
        result.append((
            day,
            overlap_start.strftime("%H:%M"),
            overlap_end.strftime("%H:%M")
        ))
    
    return result


## Create Session from Smart Time

In [ ]:
def create_smart_session(conn, host_id, guest_id):
    """
    Finds a smart time match and creates a scheduled session.
    """
    matches = find_common_time_block(host_id, guest_id, df_students, df_utc_days)
    
    if not matches:
        print("No overlapping study times found.")
        return

    # Pick the first recommended slot
    day, start_time, end_time = matches[0]
    
    cursor = conn.cursor()
    cursor.execute("""
        INSERT INTO scheduled_sessions (
            host_student_id,
            guest_student_id,
            day,
            start_time,
            end_time,
            status
        )
        VALUES (?, ?, ?, ?, ?, 'pending');
    """, (host_id, guest_id, day, start_time, end_time))

    # Notification
    cursor.execute("""
        INSERT INTO notifications (
            student_id, type, content, status, timestamp
        ) VALUES (?, ?, ?, ?, datetime('now'))
    """, (guest_id, 'invite', f"{host_id} auto-suggested a study session on {day} at {start_time}", 'unread'))
    
    conn.commit()
    print(f"✅ Smart session created for {day} {start_time}-{end_time}")
